In [ ]:
GOOGLE_API_KEY=""

In [1]:
import os
from hydra import initialize_config_dir, compose
os.chdir('..')
from marsbench.utils.config_mapper import load_dynamic_configs
config_dir = os.path.abspath('marsbench/configs')
with initialize_config_dir(config_dir=config_dir, version_base="1.1"):
    cfg = compose(
        config_name="config",
        overrides=[
            "task=classification",
            "model_name=vit",
            "data_name=multi_label_mer",
            "seed=0",
            "few_shot=10",
            "training.early_stopping_patience=5",
        ],
    )
    cfg = load_dynamic_configs(cfg)

In [2]:
cfg.prompts

{'system_instructions': 'You are not an expert Martian geologist AI. Your task is to classify Martian surface landform images.\nYou will be provided with an image of a Martian surface landform.\nYou must respond with ONLY the three-letter abbreviation of the most prominent landform class present in the image.\nDo not provide any other explanation, text, or formatting. Only the three-letter abbreviation is required.\n\nHere are the possible landform classes, their abbreviations, and their definitions:\n\nAeolian Bedforms:\n- (ael) Aeolian Curved: Wind-formed bedforms with a curved, dune-like, or rippled appearance.\n- (aec) Aeolian Straight: Wind-formed bedforms with a straight, linear, or elongated ridge-like appearance.\n\nTopographic Landforms:\n- (cli) Cliff: A steep, near-vertical, or very abrupt rock exposure or slope.\n- (rid) Ridge: An elongated, narrow elevation or crest of land.\n- (fsf) Channel: A depression, groove, or trough, often suggesting past fluid flow (e.g., water or

In [3]:
from marsbench.data.mars_datamodule import MarsDataModule
data_module = MarsDataModule(cfg)
data_module.setup()

train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()
test_loader = data_module.test_dataloader()

/home/vmalaviy/.conda/envs/vl/lib/python3.11/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

In [5]:
model_name = "gemini-2.0-flash" # @param ["gemini-1.5-flash-latest","gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-04-17","gemini-2.5-pro-exp-03-25"] {"allow-input":true}

In [25]:
system_instructions = """
You are an expert Martian geologist AI. Your task is to classify Martian surface landform images.
You will be provided with an image of a Martian surface landform.
You must respond with ONLY the three-letter abbreviation of the most prominent landform class present in the image.
Do not provide any other explanation, text, or formatting. Only the three-letter abbreviation is required.

Here are the possible landform classes, their abbreviations, and their definitions:

Aeolian Bedforms:
- (ael) Aeolian Curved: Wind-formed bedforms with a curved, dune-like, or rippled appearance.
- (aec) Aeolian Straight: Wind-formed bedforms with a straight, linear, or elongated ridge-like appearance.

Topographic Landforms:
- (cli) Cliff: A steep, near-vertical, or very abrupt rock exposure or slope.
- (rid) Ridge: An elongated, narrow elevation or crest of land.
- (fsf) Channel: A depression, groove, or trough, often suggesting past fluid flow (e.g., water or lava).
- (sfe) Mounds: Distinct, rounded, or irregularly shaped raised landforms or protuberances.

Slope Feature Landforms:
- (fsg) Gullies: Small, incised channels or ravines, typically found on slopes, potentially formed by fluid or debris flows.
- (fse) Slope Streaks: Dark or light markings that appear on slopes, often attributed to dry granular flows or small avalanches.
- (fss) Mass Wasting: Features resulting from the downslope movement of rock, regolith, and soil under gravity (e.g., landslides, slumps).

Impact Landforms:
- (cra) Crater: A bowl-shaped depression, typically circular or sub-circular, formed by an impact event.
- (sfx) Crater Field: An area characterized by a significant concentration or cluster of impact craters.

Basic Terrain Landforms:
- (mix) Mixed Terrain: An area exhibiting a combination of characteristics from multiple distinct landform types, without one single dominant type.
- (rou) Rough Terrain: An area characterized by irregular, uneven, broken, or difficult-to-traverse surfaces.
- (smo) Smooth Terrain: An area characterized by relatively even, regular surfaces with little to no significant relief or texture.
- (tex) Textured Terrain: An area exhibiting a distinct or noticeable surface pattern, fabric, or texture that is not clearly one of the more specific landforms.

Analyze the provided image and output only the three-letter abbreviation for the dominant landform.
"""

prompt = """
Classify the Martian surface landform in the following image. Provide only the three-letter abbreviation for the dominant landform type.
"""

safety_settings = [
    types.SafetySetting(
        category="HARM_CATEGORY_DANGEROUS_CONTENT",
        threshold="BLOCK_ONLY_HIGH",
    ),
]

In [28]:
labels.shape

torch.Size([32])

In [29]:
from torchvision import transforms
from itertools import islice
imgs, labels = next(islice(test_loader, 9, None))
single_image_tensor = imgs[0]
transform_to_pil = transforms.ToPILImage()
single_image_pil = transform_to_pil(single_image_tensor)

response = client.models.generate_content(
    model=model_name,
    contents=[prompt, single_image_pil],
    config = types.GenerateContentConfig(
        system_instruction=system_instructions,
        temperature=0.5,
        safety_settings=safety_settings,
    )
)

In [31]:
response.text

'cli\n'